# Подготовка данных для распознавания крокодилов и часов на изображениях

**Источник данных** - [Архив с изображениями](https://drive.google.com/file/d/1JbYmH50iRkMorFk0xNCnwC9xKiB60Mlq/view?usp=sharing), полученный по почте.

Данные разбиваются на три каталога:
- train (данные для обучения)
- val (данные для проверки)
- test (данные для тестирования)

В каждом каталоге создаются по два подкаталога, в соответсвии с названиями классов: crocodile и clock. 

Изображения в исходном каталоге сначала переименовываются в формат `class.num.png`, где `num` - последовательный номер изображения, начиная с нуля. Затем изображения копируются и сортируются в новую директорию data, разбиваясь в нужном соотношении на папки `train` для обучения, `val` для проверки в процессе обучения и `test` для оценки качества нейросети. По-умолчанию для обучения используется 70% изображений, для проверки - 15%, для тестрования также 15%. 

In [1]:
import shutil
import os

In [2]:
# Каталог с набором данных
data_dir = './clocks_crocodiles'
# Каталог с данными для обучения
train_dir = 'data/train'
# Каталог с данными для проверки
val_dir = 'data/val'
# Каталог с данными для тестирования
test_dir = 'data/test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 500

Функция создания каталога с двумя подкаталогами по названию классов: clock и crocodile

In [3]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "clock"))
    os.makedirs(os.path.join(dir_name, "crocodile"))    

Создание структуры каталогов для обучающего, проверочного и тестового набора данных

In [4]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

Функция обзора директории, чтобы получить список файлов, из которых мы будем вытягивать изображения.

In [5]:
import os
def ls_scan_path(path='.'):
    entries = [name for name in os.listdir(path)]

    files = []

    # Фильтруем entries, формируя список файлов
    for entry in entries:
        entry_path = os.path.join(path, entry)

        if os.path.isfile(entry_path):                          
            files.append(entry_path)

    # Возвращаем список файлов в директории
    return files

Используем эту функцию, чтобы переименовать все файлы из датасета в формат `class.number.jpg`. Для начала напишем ещё одну, чтобы быстро переименовывать папки.

In [6]:
def rename_all_in_dir(classname):
    class_dir = os.path.join(data_dir, classname)
    files = ls_scan_path(class_dir)

    for idx, file in enumerate(files):
        os.rename(file, os.path.join(class_dir, '{}.{}.png'.format(classname, idx)))

In [7]:
rename_all_in_dir('clock')
rename_all_in_dir('crocodile')

Функция копирования изображений в заданный каталог. Изображения часов и крокодилов копируются в отдельные подкаталоги

In [8]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    clock_dir = os.path.join(data_dir, 'clock')
    croc_dir = os.path.join(data_dir, 'crocodile')
    
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(clock_dir, "clock." + str(i) + ".png"), 
                    os.path.join(dest_dir, "clock"))
        shutil.copy2(os.path.join(croc_dir, "crocodile." + str(i) + ".png"), 
                   os.path.join(dest_dir, "crocodile"))

Расчет индексов наборов данных для обучения, приверки и тестирования

In [9]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)               


350
425


In [10]:
!ls 

Clock_or_Crocodile.ipynb  data.zip                  redist
clocks_crocodiles         dataset_preparation.ipynb trained_net
clocks_crocodiles.rar     random_clock.png
data                      random_croc.png


Копирование изображений

In [11]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

Всё готово для начала работы. Время обучить нейронную сеть.

Далее см. ClocksOrCrocs.ipynb. Рекомендуется обучать сеть в Google Colaboratory, поскольку я обучал её там.